Connected to Python 3.11.4

In [31]:
from bs4 import BeautifulSoup, element
import urllib
import urllib.request
import pandas as pd
import numpy as np
import os

In [ ]:
pages = 9
rec_count = 0
rank = []
gname = []
platform = []
year = []
genre = []
critic_score = []
user_score = []
publisher = []
developer = []
sales_na = []
sales_pal = []
sales_jp = []
sales_ot = []
sales_gl = []
# new list to hold href url_to_game
ind_game_url = []
modified_ind_game_url=[]
ind_sales_gl = []



# deselecting VGChartz Score
urlhead = 'http://www.vgchartz.com/gamedb/?page='
urltail = '&console=&region=All&developer=&publisher=&genre=&boxart=Both&ownership=Both'
urltail += '&results=1000&order=TotalSales&showtotalsales=1&showtotalsales=1&showpublisher=0'
urltail += '&showpublisher=1&showvgchartzscore=0&shownasales=1&showdeveloper=1&showcriticscore=1'
urltail += '&showpalsales=0&showpalsales=1&showreleasedate=1&showuserscore=1&showjapansales=1'
urltail += '&showlastupdate=0&showothersales=1&showgenre=1&sort=GL'



for page in range(1, pages):
    url = urlhead + str(page) + urltail
    
    
    # urllib error, no att request
    data = urllib.request.urlopen(surl, data = None).read()
    # add features = html.parser

    data = urllib.request.urlopen(url, data = None).read()
    soup = BeautifulSoup(data,features = "html.parser" )


    print(f"Page: {page}")

    # page scan for individual game links
    # <a> tags with game urls
    # swapped to https
    game_tags = list(filter(
        lambda x: 'href' in x.attrs and x.attrs['href'].startswith('https://www.vgchartz.com/game/'),
        # skipping the first link as it's being tied to navigation
        soup.find_all("a")
    ))[1:]

    for tag in game_tags:
        
        # add name to list
        gname.append(" ".join(tag.string.split()))
        print(f"{rec_count + 1} Fetch data for game {gname[-1]}")

        # get different attributes
        # traverse up the DOM tree
        data = tag.parent.parent.find_all("td")
        rank.append(np.int32(data[0].string))
        platform.append(data[3].find('img').attrs['alt'])
        publisher.append(data[4].string)
        developer.append(data[5].string)
        critic_score.append(
            float(data[6].string) if
            not data[6].string.startswith("N/A") else np.nan)
        user_score.append(
            float(data[7].string) if
            not data[7].string.startswith("N/A") else np.nan)
        sales_na.append(
            float(data[9].string[:-1]) if
            not data[9].string.startswith("N/A") else np.nan)
        sales_pal.append(
            float(data[10].string[:-1]) if
            not data[10].string.startswith("N/A") else np.nan)
        sales_jp.append(
            float(data[11].string[:-1]) if
            not data[11].string.startswith("N/A") else np.nan)
        sales_ot.append(
            float(data[12].string[:-1]) if
            not data[12].string.startswith("N/A") else np.nan)
        sales_gl.append(
            float(data[8].string[:-1]) if
            not data[8].string.startswith("N/A") else np.nan)
        release_year = data[13].string.split()[-1]
        # different format for year
        if release_year.startswith('N/A'):
            year.append('N/A')
        else:
            if int(release_year) >= 80:
                year_to_add = np.int32("19" + release_year)
            else:
                year_to_add = np.int32("20" + release_year)
            year.append(year_to_add)

        # go to every individual website to get genre info
        url_to_game = tag.attrs['href']
        site_raw = urllib.request.urlopen(url_to_game).read()
        sub_soup = BeautifulSoup(site_raw, "html.parser")
        # again, the info box is inconsistent among games so we
        # have to find all the h2 and traverse from that to the genre name
        h2s = sub_soup.find("div", {"id": "gameGenInfoBox"}).find_all('h2')
        # make a temporary tag here to search for the one that contains
        # the word "Genre"
        temp_tag = element.Tag
        for h2 in h2s:
            if h2.string == 'Genre':
                temp_tag = h2
        genre.append(temp_tag.next_sibling.string)

        # new plan
        ind_game_url.append(url_to_game)


        rec_count += 1

In [ ]:
# new tail piece for ind_game_url
new_tail = "sales"
for i in range(len(ind_game_url)):
    url = ind_game_url[i]
    last_slash_index = url.rfind('/')
    if last_slash_index != -1:
        modified_ind_game_url = url[:last_slash_index + 1] + new_tail
        ind_game_url[i] = modified_ind_game_url
        print(modified_ind_game_url)

In [25]:
# with our modified individual individual game urls (modified_ind_game_url)
# create a loop that feeds the list into soup_sales
for j in range(len(modified_ind_game_url)):
    site_sales = urllib.request.urlopen(modified_ind_game_url).read()
    soup_sales = BeautifulSoup(site_sales, "html.parser")
    br_total = soup_sales.find("td", {"id": "salesHistoryHeaderB"}).find_all('b')
    # make a temporary tag here to search for the one that contains
    # the word "Total"
    tag_sales_total = element.Tag
    for b in br_total:
        if b.string == 'Total':
            tag_sales_total = b
            ind_sales_gl.append(tag_sales_total.previous_sibling.previous_sibling.string)
            
    

In [27]:
columns = {
    'Rank': rank,
    'Name': gname,
    'Platform': platform,
    'Year': year,
    'Genre': genre,
    'Critic_Score': critic_score,
    'User_Score': user_score,
    'Publisher': publisher,
    'Developer': developer,
    'NA_Sales': sales_na,
    'PAL_Sales': sales_pal,
    'JP_Sales': sales_jp,
    'Other_Sales': sales_ot,
    'Global_Sales': sales_gl,
    'Game_URL': ind_game_url,
    'Sales_Game_URL': modified_ind_game_url,
    'Ind_Global_Sales': ind_sales_gl
}

In [ ]:
print(rec_count)
df = pd.DataFrame(columns)
print(df.columns)
df = df[[
    'Rank', 'Name', 'Platform', 'Year', 'Genre',
    'Publisher', 'Developer', 'Critic_Score', 'User_Score',
    'NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales',
    'Game_URL', 'Sales_Game_URL','Ind_Global_Sales']]


In [25]:
# change the name of the file
df.to_csv(os.getcwd()+r"\vgsales_xx.csv", sep=",", encoding='utf-8', index=False)